# ACTIVIDAD DE CLASIFICACIÓN DE TEXTO

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python". 

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

# Librerías

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Descarga de Datos

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17329808/17329808 [==============================] - 17s 1us/step


In [3]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['talk.politics.misc', 'misc.forsale', 'rec.autos', 'sci.med', 'rec.sport.hockey', 'rec.sport.baseball', 'comp.graphics', 'rec.motorcycles', 'sci.crypt', 'talk.politics.mideast', 'alt.atheism', 'talk.politics.guns', 'comp.sys.mac.hardware', 'comp.os.ms-windows.misc', 'sci.space', 'sci.electronics', 'comp.sys.ibm.pc.hardware', 'soc.religion.christian', 'comp.windows.x', 'talk.religion.misc']


In [4]:
print(data_dir)

/root/.keras/datasets/20_newsgroup


In [5]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['38723', '38769', '38256', '38932', '38250']


In [6]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "38891").read())

Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!hri.com!enterpoop.mit.edu!gatech!howland.reston.ans.net!torn!nott!bnrgate!bnr.co.uk!uknet!warwick!bham!ibm3090.bham.ac.uk!SITUNAYA
From: SITUNAYA@IBM3090.BHAM.AC.UK
Newsgroups: comp.graphics
Subject: Best FTP Viewer please.
Date: 28 Apr 93 11:50:33 BST
Organization: The University of Birmingham, United Kingdom
Lines: 5
Message-ID: <935028115033@ibm3090.bham.ac.uk>
NNTP-Posting-Host: ibm3090.bham.ac.uk

Could someone please tell me the Best FTP'able viewer available for MSDOS
I am running a 486 33mhz with SVGA monitor.
I need to look at gifs mainly and it would be advantageous if it ran
under windows...........thanks



In [7]:
#Algunos archivos de la categoria "talk.politics.misc"
fnames = os.listdir(data_dir / "talk.politics.misc")
print("Number of files in talk.politics.misc:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['179061', '179094', '176919', '178414', '178512']


In [8]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

In [17]:
samples = []
labels = []
class_names = []
class_index = 0
aux = True
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        if aux:
          print("#####################################################")
          print(lines[0:10])
          print("----------------------------------------------")
          print(lines[10:20])
          aux=False
          print("#####################################################")

        lines = lines[10:] #Aquí es donde se descartan las 10 primeras lineas de cada archivo
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
#####################################################
['Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:54169 talk.religion.misc:84359 talk.origins:41165', 'Newsgroups: alt.atheism,talk.religion.misc,talk.origins', 'Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!zaphod.mps.ohio-state.edu!uwm.edu!linac!att!att!allegra!ulysses!ulysses.att.com!mls', 'From: mls@ulysses.att.com (Michael L. Siemon)', 'Subject: Re: Ancient references to Christianity (was: Albert Sabin)', 'Summary: Use of historical sources ...', 'Message-ID: <1993Apr26.143715.22173@ulysses.att.com>', 'Date: Mon, 26 Apr 1993 14:37:15 GMT', 'References: <1r67ruINNmle@ctron-news.ctron.com> <C5ztJu.FKx@news.cso.uiuc.edu> <C62B7n.6B4@news.cso.uiuc.edu>', 'Organization: AT&T Bell Labs, Murray Hill, NJ, USA']
----------------------------------------------
['Lines: 83', '', 'In article <C62B7n.6B4@news.cso.uiuc.edu> cobb@alexia.lis.uiuc.edu', '(Mike Cobb) 

# Mezclando los datos para separarlos en Traning y Test

In [18]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples] 
val_samples = samples[-num_validation_samples:] 
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

print("Muestra del training:")
print(train_samples[0])
print("#################################################")
print("Muestra del test:")
print(val_samples[0])
print("#################################################")
print("Etiqueta del training:")
print(train_labels[0])
print("#################################################")
print("Etiqueta del test:")
print(val_labels[0])

Muestra del training:
In article <1993Apr20.151818.4319@samba.oit.unc.edu> Scott.Marks@launchpad.unc.edu (Scott Marks) writes:
>>And of course, Mike Ramsey was (at one time) the captain in Buffalo prior to
>>being traded to Pittsburgh.  Currently, the Penguins have 3 former captains
>>and 1 real captain (Lemieux) playing for them.  They rotate the A's during the
>>season (and even the C while Mario was out).  Even Troy Loney has worn the C
>>for the Pens.
>

I think that Mike Foligno was the captain of the Sabres when he
got traded to the Leafs. Also, wasn't Rick Vaive the captain of
the Leafs when he got traded to Chicago (with Steve Thomas for
Ed Olcyzk and someone). Speaking of the Leafs, I believe that
Darryl Sittler was their captain (he'd torn the "C" off his
jersey but I think he re-claimed the captaincy later on) when he
was traded to the Flyers.

Oh yeah, of course, Gretzky was the captain of the Oilers before
he was traded wasn't he? 

Gary

##################################

# Tokenización de las palabras con TextVectorization 

In [19]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [20]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [21]:
len(vectorizer.get_vocabulary())

20000

# Viendo la salida de Vectorizer

In [22]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3454, 1659,   15,    2, 8034])

In [23]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3454, 1659,   15,    2, 8034,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [24]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [25]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3454, 1659, 15, 2, 8034]

# Tokenización de los datos de entrenamiento y validación

In [26]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Creación y entrenamiento del modelo

# Utilizando el tokenizador de spacy, que ya conoces, calcula el número promedio de tokens de una muestra de 15 ficheros de la categoría ‘com.graphics’. Indica el código utilizado y el resultado obtenido

In [27]:
#Primero vamos a ver como se llaman los 15 archivos de com.graphics
import os
import random

archivos = os.listdir(data_dir/ "comp.graphics")


# Seleccionar aleatoriamente 15 archivos
archivos_seleccionados = random.sample(archivos, 15)

# Mostrar los nombres de los archivos en pantalla
for archivo in archivos_seleccionados:
    print(archivo)

# Guardar los nombres de los archivos en una variable
nombres_archivos = archivos_seleccionados


39049
38827
38393
38099
38403
38708
38340
38345
38405
38592
38928
38958
39662
38934
38462


In [28]:
import spacy
from spacy.tokens import Doc
from pathlib import Path

# Cargar el modelo de idioma en inglés
nlp = spacy.load('en_core_web_sm')

# Lista de archivos de la categoría 'comp.graphics'
archivos = nombres_archivos

# Variable para almacenar el número total de tokens
total_tokens = 0

# Procesar cada archivo
for archivo in archivos:
    # Construir la ruta completa del archivo
    ruta_archivo = data_dir / "comp.graphics" / archivo
    
    # Leer el contenido del archivo
    with open(ruta_archivo, 'r', encoding='utf-8', errors='ignore') as file:
        texto = file.read()
    
    # Crear un objeto Doc utilizando el tokenizador de spaCy
    doc = Doc(nlp.vocab, words=texto.split())
    
    # Obtener el número de tokens en el documento
    num_tokens = len(doc)
    
    # Agregar el número de tokens al total
    total_tokens += num_tokens
    
    # Mostrar la longitud del archivo
    print("Longitud del archivo", archivo, ":", num_tokens)

# Calcular el número promedio de tokens
promedio_tokens = total_tokens / len(archivos)

# Imprimir el resultado
print("Número promedio de tokens:", promedio_tokens)


Longitud del archivo 39049 : 145
Longitud del archivo 38827 : 176
Longitud del archivo 38393 : 124
Longitud del archivo 38099 : 148
Longitud del archivo 38403 : 9200
Longitud del archivo 38708 : 104
Longitud del archivo 38340 : 206
Longitud del archivo 38345 : 314
Longitud del archivo 38405 : 100
Longitud del archivo 38592 : 132
Longitud del archivo 38928 : 190
Longitud del archivo 38958 : 99
Longitud del archivo 39662 : 65
Longitud del archivo 38934 : 74
Longitud del archivo 38462 : 899
Número promedio de tokens: 798.4


# El código proporcionado lee los ficheros uno a uno y, antes de generar el catálogo de datos de entrenamiento y validación, descarta las 10 primeras líneas de cada fichero. ¿Cuál es el trozo de código en el que se realiza dicho descarte?, ¿por qué crees que se descartan dichas líneas?, ¿por qué 10 y no otro número? (1 punto)

In [29]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:] #Aquí es donde se descartan las 10 primeras lineas de cada archivo
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha



En el codigo se decartan las 10 primeras líneas con (lines = lines[10:]), asumiendo que son metadatos o encabezados que deben omitirse.



# ¿Qué se controla con el parámetro 'validation_split'?, ¿por qué se ha elegido ese valor?, ¿qué ocurre si lo modificas? 

In [30]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]


El código está utilizando el valor validation_split para controlar la proporción de muestras que se utilizarán como datos de validación en relación al total de muestras. El número específico 0.2 utilizado en este código indica que se asignará el 20% de las muestras como datos de validación, mientras que el 80% restante se utilizará como datos de entrenamiento.

La elección de este número puede variar según el conjunto de datos y los requisitos del problema. En muchos casos, se utiliza una división del 70-30 (entrenamiento-validación) o 80-20. En este caso, se ha elegido una división del 80-20.


# Imprime por pantalla un ejemplo (es decir, un elemento del array) de ‘train_samples’, ‘val_samples’, ‘train_labels’ y ‘val_labels’. A tenor de las etiquetas que se utilizan, ¿qué tarea crees que se está intentando entrenar? 

In [31]:
import random

# Imprimir un ejemplo de train_samples
print("Ejemplo de train_samples:")
indice_train = random.randint(0, len(train_samples) - 1)
print(train_samples[indice_train])
print()

# Imprimir un ejemplo de val_samples
print("Ejemplo de val_samples:")
indice_val = random.randint(0, len(val_samples) - 1)
print(val_samples[indice_val])
print()

# Imprimir un ejemplo de train_labels
print("Ejemplo de train_labels:")
indice_train_labels = indice_train
print(train_labels[indice_train_labels])
print()

# Imprimir un ejemplo de val_labels
print("Ejemplo de val_labels:")
indice_val_labels = indice_val
print(val_labels[indice_val_labels])
print()

Ejemplo de train_samples:

DAK988S@vma.smsu.edu writes:
>No....Hal McRae is the worst manager in baseball.

I haven't seen enough Royals' games to judge his tactics, so you may have
a point here.  But:

>I've never seen a guy who can waste talent like he can.  One of the best
>raw-talent staffs in the league, and he's still finding a way to lose.

IMO, the Royals don't have a chance to win the pennant even if McRae
suddenly began channeling for John McGraw.  OK, they have some decent
pitchers.  But when your offense consists of bums like Gagne and Lind
and McReynolds and McRae and an over-the-hill Brett, you're not going
to finish .500 unless McGraw brings Christy Mathewson back with him.

I'd say it is hard to evaluate a manager when all of his hitters suck.

Bob Davis	rbd@thor.ece.uc.edu


Ejemplo de val_samples:

In article <GERRY.93Apr21132149@onion.cmu.edu> gerry@cmu.edu (Gerry Roston) writes:
> 4th Amendment
> The right of the people to be secure in their persons, houses, 
> pape

# Con 'output_sequence_length' se establece un tamaño fijo para la salida de Vectorizer. ¿Por qué se necesita un tamaño fijo, y por qué se ha elegido el valor ‘200’? 

Al trabajar con modelos de aprendizaje automático, es común que la entrada y salida tengan una dimensión fija. Esto se debe a que los modelos requieren entradas de tamaño constante para realizar operaciones eficientes y tener una representación consistente de los datos.

En el caso específico de 'output_sequence_length', se establece un tamaño fijo para la salida del Vectorizer para garantizar que todas las secuencias de texto tengan la misma longitud en su representación vectorial. Esto es necesario para alimentar adecuadamente los datos a un modelo, ya que generalmente se requiere que todas las entradas tengan la misma dimensión.

El valor específico de '200' para 'output_sequence_length' fue elegido probablemente en función de la longitud promedio de las secuencias de texto en el conjunto de datos o consideraciones específicas del problema. Es común seleccionar un valor lo suficientemente grande para abarcar la mayoría de las secuencias, pero no tan grande como para causar problemas de rendimiento o memoria. Es posible que se hayan realizado experimentos previos para determinar que un tamaño de 200 era adecuado para capturar la información relevante en el contexto del problema en cuestión.

Es importante tener en cuenta que el valor de 'output_sequence_length' puede variar según el conjunto de datos y el problema específico que se esté abordando. En algunos casos, puede ser necesario ajustar este valor y realizar pruebas para encontrar el tamaño óptimo que funcione mejor para el modelo y los datos específicos. 


# Transformers

In [32]:
from tensorflow.keras import layers
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

 La clase TransformerBlock define un bloque de la arquitectura Transformer que se puede utilizar como componente en un modelo de Transformer más grande. El bloque del Transformer realiza las siguientes operaciones:

Atención multi-cabeza: Utiliza la capa MultiHeadAttention de TensorFlow para aplicar atención multi-cabeza a los datos de entrada. La atención multi-cabeza permite que el bloque capture relaciones y dependencias entre diferentes partes de la secuencia de entrada.

Capa feed-forward: Utiliza una capa Dense de TensorFlow para aplicar una transformación no lineal (usualmente una función de activación ReLU) a la salida de la atención multi-cabeza. La capa feed-forward introduce no linealidad y ayuda al bloque a aprender representaciones más complejas de los datos.

Normalización: Utiliza las capas LayerNormalization de TensorFlow para realizar la suma residual y la normalización en la salida de la atención multi-cabeza y de la capa feed-forward. La normalización ayuda a estabilizar el aprendizaje y facilita el flujo de los gradientes durante la retropropagación.

Dropout: Utiliza las capas Dropout de TensorFlow para aplicar regularización mediante la eliminación aleatoria de conexiones durante el entrenamiento. El dropout ayuda a prevenir el sobreajuste y mejora la generalización del modelo.

En conjunto, el bloque del Transformer captura patrones y relaciones en los datos de entrada a través de la atención multi-cabeza, aplica transformaciones no lineales con la capa feed-forward y utiliza normalización y dropout para regularizar y estabilizar el proceso de aprendizaje. Este bloque se puede repetir varias veces en un modelo de Transformer más grande para construir arquitecturas más profundas y complejas que se utilizan en tareas de procesamiento del lenguaje natural y otros dominios.

In [33]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

La clase TokenAndPositionEmbedding implementa una capa de embedding que agrega información de tokens y posiciones a través de incrustaciones. Esto permite que el modelo capture tanto el contenido (significado) como la estructura posicional de la secuencia de entrada. Al combinar estas dos fuentes de información, el modelo puede comprender mejor las relaciones y dependencias entre los tokens en la secuencia, lo que es especialmente útil en tareas de procesamiento del lenguaje natural y otros dominios donde la estructura posicional es importante para comprender el significado global de la secuencia.

In [34]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
num_tokens = len(voc) + 2

maxlen = 200 
vocab_size = num_tokens

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(len(class_names), activation="softmax")(x)

modeloTransformers = keras.Model(inputs=inputs, outputs=outputs)

Se definen varios parámetros relacionados con la arquitectura del modelo:

embed_dim representa la dimensión de las incrustaciones (embeddings) para cada token en el texto.
num_heads especifica el número de cabezas de atención utilizadas en la capa de atención multi-cabeza del Transformer.
ff_dim es el tamaño de la capa oculta en la red de retroalimentación (feed-forward) dentro del bloque Transformer.
num_tokens es el número total de tokens en el vocabulario, incluyendo tokens especiales (como el token de inicio y el token de padding).
Se definen los tamaños de las secuencias de entrada y el tamaño del vocabulario:

maxlen es la longitud máxima de las secuencias de entrada (texto). Se establece en 200 tokens en este caso.
vocab_size es el tamaño del vocabulario utilizado para la incrustación de tokens y se establece como num_tokens.
Se define una capa de entrada (inputs) que espera una secuencia de longitud maxlen de tokens.

Se crea una capa de incrustación (embedding_layer) utilizando la clase TokenAndPositionEmbedding que definiste anteriormente. Esta capa combina la información de tokens y posiciones en la secuencia de entrada.

La secuencia de entrada se pasa a través de la capa de incrustación (x = embedding_layer(inputs)).

Se crea un bloque Transformer (transformer_block) utilizando la clase TransformerBlock que definiste anteriormente. Este bloque aplica la atención multi-cabeza y la red de retroalimentación para capturar relaciones y aprender representaciones más complejas de los datos.

La secuencia de incrustaciones se pasa a través del bloque Transformer (x = transformer_block(x)).

Se aplica una operación de pooling global promedio (layers.GlobalAveragePooling1D()) para resumir las características de la secuencia en un vector de características de longitud fija.

Se aplica una capa de dropout (layers.Dropout(0.1)) para regularizar el modelo.

Se agrega una capa densa (layers.Dense(20, activation="relu")) con una función de activación ReLU para introducir no linealidad y aprender representaciones más complejas.

Se aplica una capa de dropout adicional.

Finalmente, se agrega una capa densa de salida (layers.Dense(len(class_names), activation="softmax")) con una función de activación softmax para obtener las probabilidades de pertenencia a cada clase de salida.

En resumen, este código define una arquitectura de modelo basada en Transformer para clasificar texto, que combina capas de incrustación de tokens y posiciones, capas de atención multi-cabeza y capas densas para aprender representaciones y realizar la clasificación.

In [35]:
modeloTransformers.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloTransformers.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloTransformers.summary())

Epoch 1/20
125/125 [==============================] - 27s 147ms/step - loss: 2.9039 - acc: 0.0949 - val_loss: 2.6165 - val_acc: 0.1968
Epoch 2/20
125/125 [==============================] - 13s 106ms/step - loss: 2.3960 - acc: 0.2315 - val_loss: 1.9428 - val_acc: 0.4004
Epoch 3/20
125/125 [==============================] - 12s 93ms/step - loss: 1.8361 - acc: 0.4004 - val_loss: 1.5716 - val_acc: 0.4684
Epoch 4/20
125/125 [==============================] - 11s 84ms/step - loss: 1.4035 - acc: 0.5424 - val_loss: 1.2118 - val_acc: 0.6049
Epoch 5/20
125/125 [==============================] - 9s 70ms/step - loss: 1.1372 - acc: 0.6175 - val_loss: 0.9449 - val_acc: 0.7002
Epoch 6/20
125/125 [==============================] - 6s 46ms/step - loss: 0.9549 - acc: 0.6851 - val_loss: 1.0764 - val_acc: 0.6507
Epoch 7/20
125/125 [==============================] - 6s 50ms/step - loss: 0.8445 - acc: 0.7209 - val_loss: 0.8700 - val_acc: 0.7247
Epoch 8/20
125/125 [==============================] - 6s 45ms/s

In [37]:
val_loss, val_accuracy = modeloTransformers.evaluate(x_val, y_val)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

125/125 [==============================] - 1s 5ms/step - loss: 1.1277 - acc: 0.7674
Validation Loss: 1.1276781558990479
Validation Accuracy: 0.7674418687820435


# Red Neuronal Clásica

In [ ]:
modeloClasico = keras.models.Sequential()
modeloClasico.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloClasico.add(keras.layers.Flatten())
modeloClasico.add(keras.layers.Dense(512, activation='relu'))
modeloClasico.add(keras.layers.Dropout(0.3))
modeloClasico.add(keras.layers.Dense(20, activation='softmax'))

In [ ]:
modeloClasico.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloClasico.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloClasico.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloClasico.summary())

Epoch 1/20
125/125 [==============================] - 21s 163ms/step - loss: 2.8953 - acc: 0.0972 - val_loss: 2.6500 - val_acc: 0.1568
Epoch 2/20
125/125 [==============================] - 14s 107ms/step - loss: 2.2651 - acc: 0.2696 - val_loss: 1.9869 - val_acc: 0.3246
Epoch 3/20
125/125 [==============================] - 11s 92ms/step - loss: 1.5958 - acc: 0.4891 - val_loss: 1.5678 - val_acc: 0.4734
Epoch 4/20
125/125 [==============================] - 7s 52ms/step - loss: 1.1163 - acc: 0.6531 - val_loss: 1.3060 - val_acc: 0.5459
Epoch 5/20
125/125 [==============================] - 8s 61ms/step - loss: 0.7759 - acc: 0.7729 - val_loss: 1.1210 - val_acc: 0.6172
Epoch 6/20
125/125 [==============================] - 5s 37ms/step - loss: 0.5413 - acc: 0.8430 - val_loss: 1.0155 - val_acc: 0.6542
Epoch 7/20
125/125 [==============================] - 6s 51ms/step - loss: 0.3921 - acc: 0.8908 - val_loss: 0.9986 - val_acc: 0.6714
Epoch 8/20
125/125 [==============================] - 4s 32ms/st

In [ ]:
val_loss, val_accuracy = modeloClasico.evaluate(x_val, y_val)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

125/125 [==============================] - 0s 2ms/step - loss: 1.1981 - acc: 0.7064
Validation Loss: 1.198068618774414
Validation Accuracy: 0.7064266204833984


# Indica cuál es la precisión del modelo en el conjunto de datos de entrenamiento y en el conjunto de datos de validación. ¿Qué interpretación puedes dar? Haz en este punto un análisis comparativo de los dos modelos ejecutados. 

# Modelo Transformers:

125/125 [==============================] - 1s 5ms/step - loss: 0.9803 - acc: 0.7882

Validation Loss: 0.980324387550354

Validation Accuracy: 0.7881970405578613

# Modelo Clásico:

125/125 [==============================] - 0s 2ms/step - loss: 1.1981 - acc: 0.7064

Validation Loss: 1.198068618774414

Validation Accuracy: 0.7064266204833984


# 7.	En la parte final del código se hace un análisis cualitativo de la salida. Explica el funcionamiento de este análisis e interpreta los resultados. Haz también en este punto un análisis comparativo de los dos modelos ejecutados. 

# Evaluación Transformers

In [ ]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloTransformers(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 294ms/step


'comp.graphics'

In [ ]:
probabilities = end_to_end_model.predict(
    [["politics and federal courts law that people understand with politician and elects congressman"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 56ms/step


'talk.politics.mideast'

In [ ]:
probabilities = end_to_end_model.predict(
    [["we are talking about religion"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 34ms/step


'soc.religion.christian'

# Evaluación Red Clásica

In [ ]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloClasico(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 117ms/step


'comp.graphics'

In [ ]:
probabilities = end_to_end_model.predict(
    [["politics and federal courts law that people understand with politician and elects congressman"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 31ms/step


'talk.politics.misc'

In [ ]:
probabilities = end_to_end_model.predict(
    [["we are talking about religion"]]
)

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 47ms/step


'talk.religion.misc'

# 8.	Explica algunas de las limitaciones que puedes encontrar al modelo entrenado.

A)	Requisitos de recursos computacionales y memoria: Los modelos Transformer se caracterizan por requerir una alta capacidad computacional y grandes cantidades de memoria debido a su arquitectura basada en atención y procesamiento paralelo de secuencias largas. Esto puede dificultar su implementación y entrenamiento en dispositivos con recursos limitados, lo que hace necesario el uso de supercomputadoras para desplegar eficientemente este tipo de modelos.

B)	Dependencia de datos etiquetados y recursos de entrenamiento: Los modelos Transformer generalmente necesitan una gran cantidad de datos etiquetados para un entrenamiento efectivo. La obtención y etiquetado de conjuntos de datos de alta calidad puede resultar costoso y laborioso, especialmente en dominios especializados o con recursos limitados. Además, la falta de contexto semántico puede agravar este desafío.

C)	Limitaciones en la interpretación: Aunque los modelos Transformer pueden lograr un alto rendimiento en diversas tareas, presentan limitaciones en cuanto a su capacidad para interpretar y explicar los resultados. Debido a la naturaleza de su arquitectura y al procesamiento a nivel de token, comprender cómo se llega a ciertas predicciones o qué características específicas se consideran relevantes puede resultar difícil.

D)	Tratamiento de secuencias de longitud variable: Aunque los modelos Transformer pueden manejar secuencias de longitud variable, a menudo requieren técnicas adicionales como el uso de tokens especiales de inicio y fin, así como el relleno de secuencias más cortas para igualar la longitud. Estas técnicas pueden introducir ruido y dificultar el manejo de secuencias extremadamente largas o cortas. Por lo tanto, es necesario aplicar técnicas de procesamiento, como la tokenización o lematización, para obtener información relevante.

E)	Captura de relaciones a largo plazo: A pesar de los avances logrados por los modelos Transformer en la captura de relaciones a largo plazo en secuencias, aún pueden enfrentar dificultades para capturar dependencias extremadamente prolongadas. En ocasiones, pueden perder información temporal o contextual en secuencias complejas, lo que puede resultar en un desequilibrio de clases y una precisión más baja para las clases menos representadas.

F)	Sensibilidad a datos de entrenamiento sesgados: Al igual que otros modelos de aprendizaje automático, los modelos Transformer pueden verse afectados por sesgos presentes en los datos de entrenamiento. Si los datos de entrenamiento están sesgados hacia ciertas clases o contienen sesgos culturales o sociales, el modelo puede aprender y amplificar esos sesgos en sus predicciones.  


# 9.	¿Qué sería necesario para que este modelo pueda interpretar textos en español? 

Para entrenar un modelo Transformer en español, es necesario contar con un conjunto de datos de entrenamiento en español que capture las características y patrones específicos del lenguaje. Estos datos deben reflejar las diferencias lingüísticas entre el español y otros idiomas, como la concordancia gramatical y las reglas de género para los sustantivos.

Un enfoque común es utilizar un modelo de lenguaje pre-entrenado en español como punto de partida. Estos modelos han sido previamente entrenados en grandes cantidades de datos en español para aprender la estructura del lenguaje. Ejemplos de modelos de lenguaje pre-entrenados en español incluyen BERT en español, XLM-R en español y otros disponibles a través de la biblioteca de HuggingFace.

Además, se puede realizar un ajuste fino (fine-tuning) del modelo utilizando conjuntos de datos más pequeños y específicos en español, adaptándolo a tareas o dominios particulares como clasificación de texto o traducción. Para lograr un procesamiento del lenguaje natural más efectivo en español, también se requieren recursos lingüísticos específicos del español, como modelos de tokenización, etiquetado gramatical, diccionarios de palabras y recursos de lematización. Estos recursos ayudan al modelo a comprender y procesar el español de manera precisa.

Es importante llevar a cabo una evaluación continua y realizar ajustes en el modelo a medida que se utilice para interpretar textos en español. Esto implica corregir errores, ajustar los hiperparámetros del modelo y recopilar comentarios de los usuarios para mejorar su capacidad de interpretación en contextos específicos del español.
